In [1]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *
from tbdynamics.utils import build_contact_matrix
import plotly.graph_objects as go
from general_utils.parameter_utils import load_param_info
from general_utils.calibration_utils import round_sigfig
from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
    save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50]

In [3]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5
}

In [4]:
matrix = build_contact_matrix()

In [5]:
tb_model = model.build_model(
    compartments,
    infectious_compartments,
    latent_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    matrix,
    fixed_parameters
)

TypeError: build_model() missing 1 required positional argument: 'tex_doc'

In [6]:
tb_model.run(params)

In [7]:
tb_model.get_outputs_df()

,susceptibleXage_0,susceptibleXage_5,susceptibleXage_15,susceptibleXage_35,susceptibleXage_50,early_latentXage_0,early_latentXage_5,early_latentXage_15,early_latentXage_35,early_latentXage_50,...,on_treatmentXage_35Xorgan_smear_negative,on_treatmentXage_35Xorgan_extrapulmonary,on_treatmentXage_50Xorgan_smear_positive,on_treatmentXage_50Xorgan_smear_negative,on_treatmentXage_50Xorgan_extrapulmonary,recoveredXage_0,recoveredXage_5,recoveredXage_15,recoveredXage_35,recoveredXage_50
1800.0,45468.751439,45468.751439,45468.751439,45468.751439,45468.751439,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1801.0,43449.793912,49336.200918,47742.505398,44380.262668,42249.029760,0.051564,0.142818,0.833813,0.966561,1.096797,...,0.002632,0.001850,0.003729,0.002577,0.001804,0.001618,0.002166,0.001976,0.001853,0.001804
1802.0,41878.700824,52477.062934,50237.557766,43485.898575,39392.497914,0.071791,0.220817,1.346890,1.482029,1.620565,...,0.004410,0.002972,0.007138,0.004266,0.002846,0.004900,0.008371,0.007320,0.006474,0.006183
1803.0,40673.487220,55040.740696,52878.603003,42779.563136,36862.486200,0.103025,0.346207,2.124031,2.204382,2.312761,...,0.006211,0.003975,0.011386,0.005934,0.003744,0.008944,0.018677,0.016309,0.013676,0.012796
1804.0,39768.628406,57147.676297,55607.076264,42252.296941,34626.674603,0.150229,0.540456,3.361239,3.291152,3.300843,...,0.008576,0.005210,0.017123,0.008056,0.004820,0.013858,0.034008,0.030229,0.024160,0.022073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016.0,226767.409688,422899.915185,421464.698488,139903.060403,48468.538673,66.270454,326.016823,6766.120747,4465.380856,2851.367439,...,288.575364,203.497408,416.094421,214.942212,151.802687,416.844556,9484.782238,48990.143830,40348.615576,29665.446471
2017.0,226664.043081,424036.940250,431170.445216,145721.433965,51588.008946,67.180867,323.175691,6678.737941,4534.412130,2923.320941,...,288.297312,201.890396,421.918107,217.451073,152.580393,395.670569,8901.982560,48020.802581,40356.793404,30179.451573
2018.0,226823.902118,425520.041598,443365.940710,152931.363577,55367.132792,35.022037,188.686116,4669.825691,3302.147353,2150.731833,...,490.001904,406.297955,630.798379,371.393733,308.117154,407.642703,8491.378888,47507.442633,40719.592462,30968.672378
2019.0,227116.984899,427361.435005,458026.794261,161674.592968,59919.748668,25.757460,118.397436,3050.988379,2253.940308,1485.070044,...,449.165685,372.242910,633.604961,346.960688,287.591572,416.604748,8145.484265,47184.098193,41241.467368,31900.274105


In [8]:
priors = [
    esp.UniformPrior("start_population_size", (150000, 300000)),
    esp.UniformPrior("contact_rate", (0.0001, 0.02)),
    #UniformPrior("infectious_seed", [100, 2000]),
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    esp.UniformPrior("cdr_adjustment", (0.6, 1.0)),\
    esp. UniformPrior("rr_progression_diabetes", (2.0, 6.0))
    # UniformPrior("progression_multiplier", (0.1, 2.0)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    # UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    # UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    # UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    # UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # UniformPrior("rr_progression_diabetes", [1, 10]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

targets = [
    est.NegativeBinomialTarget('notifications', notif, dispersion_param=50),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [9]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 150000.0 (within bound (150000.0, 300000.0)
   contact_rate: 0.01153 (within bound (0.0001, 0.02)
   rr_infection_latent: 0.2015 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.201 (within bound (0.1, 0.5)
   smear_positive_death_rate: 0.3643 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.02483 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.2034 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.2083 (within bound (0.073, 0.20900000000000002)
   cdr_adjustment: 0.602 (within bound (0.6, 1.0)
   rr_progression_diabetes: 5.999 (within bound (2.0, 6.0)


In [10]:
params.update(map_params)
tb_model.run(params)

In [11]:
tb_model.run(parameters=params)
derived_df_0 = tb_model.get_derived_outputs_df()

In [ ]:
derived_df_0

In [12]:
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [13]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()
total_fig_name = "total.jpg"
fig2_3.write_image(SUPPLEMENT_PATH /total_fig_name)

NameError: name 'SUPPLEMENT_PATH' is not defined

In [ ]:
add_element_to_document("Outputs",FigElement(total_fig_name, caption="Notifications"), doc_sections)

In [ ]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()
notif_fig_name = "notifications.jpg"
notif_plot.write_image(SUPPLEMENT_PATH / notif_fig_name)

In [ ]:
add_element_to_document("Outputs",FigElement(notif_fig_name, caption="Notifications"), doc_sections)

In [ ]:
prev_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
prev_plot.show()
# prev_plot.write_image(str(SUPPLEMENT_PATH) + "/prevalance.jpg")

In [14]:
inci_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="incidence",
)
inci_plot.show()
# inci_plot.write_image(str(SUPPLEMENT_PATH) + "/incidence.jpg")

In [15]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()
# latent_plot.write_image(str(SUPPLEMENT_PATH) + "/latent.jpg")

NameError: name 'SUPPLEMENT_PATH' is not defined

In [ ]:
supplement = generate_doc("Supplemental Appendix", "austcovid")
compile_doc(doc_sections, supplement)

In [ ]:
# iterations = 20000
# burn_in = 2000
# n_chains = 20
# if new_calibration:
#     with pm.Model() as pm_model:
#         variables = epm.use_model(calibration_model)
#         idata_raw = pm.sample(step=[pm.DEMetropolis(variables)], draws=iterations, tune=0, cores=16, chains=n_chains)
#     idata_raw.to_netcdf(OUTPUT_PATH / "calibration_out.nc")
# else:
#     idata_raw = az.from_netcdf(OUTPUT_PATH / "calibration_out.nc")

# idata = idata_raw.sel(draw=range(burn_in, iterations))  # Discard burn-in

In [ ]:
idata_raw.posterior.isel(draw=0).to_dataframe()

In [ ]:
(idata.sample_stats.accepted.sum(axis=1) / idata.sample_stats.coords["draw"].size).to_dataframe()